In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox
from IPython.display import display, clear_output

global_grayscale_image = None

def simple_threshold_segmentation(image, threshold=128):
    binary_image = np.zeros_like(image, dtype=np.uint8)

    for r in range(image.shape[0]):
        for c in range(image.shape[1]):
            if image[r, c] < threshold:
                binary_image[r, c] = 1
    return binary_image

def erosion(image, se_size):
    rows, cols = image.shape
    eroded_image = np.zeros_like(image, dtype=np.uint8)
    padding = se_size // 2

    for r in range(padding, rows - padding):
        for c in range(padding, cols - padding):
            neighborhood = image[r-padding : r+padding+1, c-padding : c+padding+1]
            if np.all(neighborhood == 1):
                eroded_image[r, c] = 1
    return eroded_image

def dilation(image, se_size):
    rows, cols = image.shape
    dilated_image = np.zeros_like(image, dtype=np.uint8)
    padding = se_size // 2

    for r in range(padding, rows - padding):
        for c in range(padding, cols - padding):
            neighborhood = image[r-padding : r+padding+1, c-padding : c+padding+1]
            if np.any(neighborhood == 1):
                dilated_image[r, c] = 1
    return dilated_image

def opening(image, se_size):
    eroded = erosion(image, se_size)
    opened = dilation(eroded, se_size)
    return opened

def closing(image, se_size):
    dilated = dilation(image, se_size)
    closed = erosion(dilated, se_size)
    return closed

def process_and_display():

    global global_grayscale_image

    FIXED_THRESHOLD = 128
    OPENING_SE_SIZE = 3
    CLOSING_SE_SIZE = 3

    if global_grayscale_image is None:
        print("Upload an image")
        return

    segmented_image = simple_threshold_segmentation(global_grayscale_image, FIXED_THRESHOLD)
    opened_image = opening(segmented_image, OPENING_SE_SIZE)
    closed_image = closing(opened_image, CLOSING_SE_SIZE)

    plt.figure(figsize=(20, 7))
    plt.gray()

    plt.subplot(1, 3, 1)
    plt.imshow(segmented_image, interpolation='none')
    plt.title(f"Without Morphology\n(Raw Segmentation @ Threshold {FIXED_THRESHOLD})", fontsize=14)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(opened_image, interpolation='none')
    plt.title(f"After Opening\n(Removes specks < {OPENING_SE_SIZE}x{OPENING_SE_SIZE})", fontsize=14)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(closed_image, interpolation='none')
    plt.title(f"After Closing\n(Fills holes < {CLOSING_SE_SIZE}x{CLOSING_SE_SIZE})", fontsize=14)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Your Image'
)

output_ui = widgets.Output()

def on_upload_change(change):
    global global_grayscale_image

    if not uploader.value:
        return

    filename = list(uploader.value.keys())[0]
    file_content = uploader.value[filename]['content']

    with output_ui:
        clear_output(wait=True)

        nparr = np.frombuffer(file_content, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)

        if image is None:
            print(f"Error: Could not read '{filename}'.")
            return

        if len(image.shape) > 2 and image.shape[2] in [3, 4]:
            global_grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            print(f"Uploaded and converted '{filename}' to grayscale.")
        else:
            global_grayscale_image = image
            print(f"Uploaded grayscale image '{filename}'.")

        process_and_display()

    uploader.value.clear()
    uploader._counter = 0

uploader.observe(on_upload_change, names='_counter')
ui_layout = VBox([uploader, output_ui])

display(ui_layout)